# Kybernetes 2 - An Analysis

This notebook is an experiment on analyzing the log data from Kybernetes 2 in a visual way using Jupyter Notebooks and Bokeh

### Processing Kybernetes 2 Log Data

Honestly the log format from the robot is garbage. I need to actually define a sensor recording format. A snippet of log data from the robot:

#### Define a function to rip GPS data from a log file

In [2]:
import re

def read_gps_data(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()

    gps_lines = [line for line in lines if "gps = Position(" in line]
    regex = r"\[(?P<timestamp>-?\d+\.\d+)\] gps = Position\(latitude=(?P<latitude>-?\d+\.\d+), longitude=(?P<longitude>-?\d+\.\d+), declination=(?P<declination>-?\d+\.\d+)\)"
    parsed = [re.search(regex, line) for line in gps_lines]
    positions = [(float(m.group('timestamp')), float(m.group('latitude')), float(m.group('longitude'))) for m in parsed]
    return positions

In [3]:
positions = read_gps_data('logs/robomagellan.2024.robogames.run1.txt')

#### Read Google Maps API Key from OS environment

If you don't have a Google Maps API Key, create a project though the Google Cloud Admin console

https://console.cloud.google.com/google/maps-apis/

In [4]:
import os

API_KEY = os.environ['GOOGLE_API_KEY']

#### Import Bokeh

In [5]:
from bokeh.io import output_notebook, push_notebook, show
from bokeh.models import GMapOptions, ColumnDataSource, Line, Circle
from bokeh.plotting import gmap

output_notebook()

Loading BokehJS ...

#### Define a function to show a map with all of the position data

In [6]:
def display_map_with_positions(positions, name="Kybernetes 2 GPS Samples", zoom=20, api_key=API_KEY):
    # Rip latitude and longitude from position tuple
    lats = [position[1] for position in positions]
    lons = [position[2] for position in positions]

    # Find the center point of data
    lat = (max(lats) + min(lats)) / 2.0
    lon = (max(lons) + min(lons)) / 2.0

    # Draw each position as a circle on the map
    map_options = GMapOptions(lat=lat, lng=lon, map_type='satellite', tilt=0, zoom=zoom)
    plot = gmap(api_key, map_options, title=name, width=800, height=600)
    plot.circle(lons, lats, size=10, alpha=0.05, color='blue')
    #plot.line(lons, lats, line_width=8, color='blue', alpha=0.5)
    show(plot)

def display_map_from_log(filename, zoom=20):
    positions = read_gps_data(filename)
    display_map_with_positions(positions, name=f'Kybernetes 2 GPS Samples - {filename}', zoom=zoom)

### Robogames 2024 - Run 1

In [7]:
display_map_from_log('logs/robomagellan.2024.robogames.run1.txt')

### Robogames 2024 - Run 2

In [8]:
display_map_from_log('logs/robomagellan.2024.robogames.run2.txt')

### Robogames 2023 - Run 1

In [10]:
display_map_from_log('logs/robomagellan.2023.robogames.run1.txt', zoom=19)

### Botnic 2024 - Run 3

In [13]:
display_map_from_log('logs/robomagellan.2024.botnic.run3.txt', zoom=19)

### Botnic Spring 2025 - Testing

In [15]:
display_map_from_log('logs/robomagellan.2025.botnic.spring.run10.txt', zoom=19)

## Animated Maps

### Define a function to draw an animated map

In [16]:
import time
import sleep_until

def display_dynamic_map_with_positions(positions, name="Kybernetes 2 GPS Samples", zoom=20, api_key=API_KEY):
    # Rip latitude and longitude from position tuple
    times = [position[0] - (positions[0][0] + 0) for position in positions]
    lats = [position[1] for position in positions]
    lons = [position[2] for position in positions]

    # Find the center point of data
    lat = (max(lats) + min(lats)) / 2.0
    lon = (max(lons) + min(lons)) / 2.0

    # Draw each position as a circle on the map
    map_options = GMapOptions(lat=lat, lng=lon, map_type='satellite', tilt=0, zoom=zoom)
    plot = gmap(api_key, map_options, title=name, width=1024, height=768)

    # stuff for animation
    source = ColumnDataSource(data=dict(lat=[], lon=[]))
    line = Line(x="lon", y="lat", line_width=4, line_color="red")
    plot.add_glyph(source, line)

    handle = show(plot, notebook_handle=True)
    initial = time.time()

    for i in range(1, len(times)):
        source.data = dict(lat=lats[:i],lon=lons[:i])
        push_notebook(handle=handle)
        sleep_until.sleep_until(initial + times[i])

def display_dynamic_map_from_log(filename, zoom=20, offset=0):
    positions = read_gps_data(filename)
    display_dynamic_map_with_positions(positions[offset:], name=f'Kybernetes 2 GPS Samples - {filename}', zoom=zoom)

### Robogames 2023 - Run 1 - Animation

In [20]:
display_dynamic_map_from_log('logs/robomagellan.2023.robogames.run1.txt', zoom=20, offset=0)

#### Botnic Spring 2025 - Run 1 - Animation

In [17]:
display_dynamic_map_from_log('logs/robomagellan.2025.botnic.spring.run10.txt', zoom=19, offset=0)